In [12]:
%load_ext autoreload
%autoreload 2

import os

# Read the 3D models from the YCB dataset
models_paths = os.listdir('ycb')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import time
from scipy.spatial.transform import Rotation as R
import mujoco
import mujoco.viewer
import mujoco.renderer
import mujoco.glfw
import numpy as np
from sensors import Sensors
from point_cloud import PointCloud
import mediapy as media

%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [3]:
models = [models_paths[i] for i in range(3)]

In [50]:
def new_xml(model, quat):
    xml =  f"""
    <mujoco>
        <asset>
            <mesh name="{model}" file="ycb/{model}" scale="1 1 1"/>
        </asset>
        <worldbody>
            <light name="top" pos="0 0 1"/>
            <geom pos="0 0 0" type="mesh" contype="0" conaffinity="0" group="1" density="0" mesh="{model}"
            quat="{quat[0]} {quat[1]} {quat[2]} {quat[3]}"
            />
            
            # Cameras
            <camera name="camera1" pos="-1 0 0" xyaxes="0 -0.5 0 -0.01 0 2" mode="fixed" fovy="60" />
            <camera name="camera2" pos="1 0 0" xyaxes="0 0.5 0 0.01 0 2" mode="fixed" fovy="60" />
            <camera name="camera3" pos="-1 0 0" xyaxes="0 -0.5 0 -0.01 0 2" mode="fixed" fovy="60" />
            <camera name="camera4" pos="-1 0 0" xyaxes="0 -0.5 0 -0.01 0 2" mode="fixed" fovy="60" />
            <camera name="camera5" pos="-1 0 0" xyaxes="0 -0.5 0 -0.01 0 2" mode="fixed" fovy="60" />
            <camera name="camera6" pos="-1 0 0" xyaxes="0 -0.5 0 -0.01 0 2" mode="fixed" fovy="60" />            
  
        </worldbody>
    </mujoco>
    """

    return xml

In [46]:
# Random quaternion
quat = np.random.random(4)
quat = quat / np.linalg.norm(quat)

m = mujoco.MjModel.from_xml_string(new_xml("003_cracker_box.stl", quat))
d = mujoco.MjData(m)

# Launching the viewer
with mujoco.viewer.launch_passive(m, d) as viewer:


  # Close the viewer automatically after 30 wall-seconds.
  start = time.time()


  while viewer.is_running():

    step_start = time.time()


    # mj_step can be replaced with code that also evaluates
    # a policy and applies a control signal before stepping the physics.
    mujoco.mj_step(m, d)
    
    # Pick up changes to the physics state, apply perturbations, update options from GUI.
    viewer.sync()


    # Rudimentary time keeping, will drift relative to wall clock.
    time_until_next_step = abs(m.opt.timestep - (time.time() - step_start))

    current_time = time.time() - start
    while d.time < current_time:
      mujoco.mj_step(m, d)

    if time_until_next_step > 0:
      time.sleep(time_until_next_step)

In [52]:
# Creating instance of classes for data collection
sensors = Sensors()
point_c = PointCloud((480, 640), 60, downsample=5)

for model in models:
    # Generating random quaternion
    quat = np.random.random(4)
    quat = quat / np.linalg.norm(quat)

    # Generating new xml
    xml = new_xml(model, quat)

    # Create the model and data objects.
    m = mujoco.MjModel.from_xml_string(xml)
    d = mujoco.MjData(m)

    # Make renderer, render and show the pixels
    depth = mujoco.Renderer(m,480, 640); depth.enable_depth_rendering()
    sgmnt = mujoco.Renderer(m,480, 640); sgmnt.enable_segmentation_rendering()
    
    # Doing initial loading
    mujoco.mj_step(m, d)

    #TODO: Get point cloud from n out of m random cameras
    n_cameras = m.cam_user.shape[0]

    # This can be obtained from the model at the beginning of the simulation
    # Or thery can be retrieved at any time during the simulation
    # Get the position and rotation of the cameras
    poses = []
    rotations = []
    for i in range(m.cam_user.shape[0]):
        # Get the position of the camera
        pos = m.cam_pos[i] + m.body(m.cam_bodyid[i]).pos
        poses.append(pos)

        # Get the rotation of the camera
        angle = R.from_quat(m.body(m.cam_bodyid[i]).quat).as_euler('xyz', degrees=True)
        rotations.append(angle[0])

    # Renderer with depth enabled
    depth_frames = [*sensors.get_depth_image_matrices(m, d, depth).values()]

    # Renderer with segmentation enabled
    sgmnt_frames = [*sensors.get_segment_image_matrices(m, d, sgmnt).values()]

    for i in range(len(sgmnt_frames)):
        plt.imshow(sgmnt_frames[i])

    # Get the point cloud from the segmented map
    points = point_c.get_segmented_map(depth_frames, sgmnt_frames, poses, rotations, m)

    # 
    for key in points.keys():
        mat = points[key]

        # Create a 3D figure
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        fig.suptitle(key)

        # Define the x, y, z coordinates of the point cloud
        x = mat[:, 0]
        y = mat[:, 1]
        z = mat[:, 2]

        # Plot the point cloud data
        ax.scatter(x, y, z, s=1)

        # Set the axis labels
        ax.set_xlabel('X Label')
        ax.set_ylabel('Y Label')
        ax.set_zlabel('Z Label')

        # Show the plot
        plt.show()
        

    print(model)
    

071_nine_hole_peg_test.stl
038_padlock.stl
065-e_cups.stl
